# 🎯 Classification d'Intentions - Neural Chat Engine

## Objectif
Développer un modèle de classification d'intentions multi-domaines utilisant DistilBERT pour notre chatbot IA avancé.

## Domaines couverts
- 🔧 **Assistance technique** (documentation, debugging)
- 🛍️ **E-commerce** (recommandations, support client)
- 👥 **RH** (screening, questions d'emploi)
- 🌍 **Multilingue** (traduction contextuelle)

## Architecture
- **Modèle base** : DistilBERT (distilbert-base-multilingual-cased)
- **Classification** : Multi-labels avec confidence scoring
- **Optimisation** : Fine-tuning avec techniques avancées

## 📚 Installation et Imports

In [ ]:
# Installation des dépendances (si nécessaire)
# !pip install transformers torch datasets scikit-learn matplotlib seaborn wandb

import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Ajout du path du projet
sys.path.append('../src')

# Imports essentiels
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import json
from typing import List, Dict, Tuple, Optional

# Transformers & ML
from transformers import (
    DistilBertTokenizer, 
    DistilBertForSequenceClassification,
    TrainingArguments, 
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict
from sklearn.metrics import (
    accuracy_score, 
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)
from sklearn.model_selection import train_test_split

# Configuration du device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Device utilisé: {device}")
print(f"🔢 PyTorch version: {torch.__version__}")

# Configuration pour la reproductibilité
torch.manual_seed(42)
np.random.seed(42)

# Style des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 🗂️ Chargement et Exploration des Données

In [ ]:
# Définition des intentions par domaine
INTENT_CATEGORIES = {
    'technical': [
        'code_debug', 'documentation_request', 'api_usage', 
        'error_explanation', 'best_practices', 'architecture_advice'
    ],
    'ecommerce': [
        'product_search', 'price_inquiry', 'recommendation_request',
        'order_status', 'return_policy', 'payment_help'
    ],
    'hr': [
        'job_inquiry', 'interview_preparation', 'salary_negotiation',
        'career_advice', 'skill_assessment', 'company_culture'
    ],
    'multilingual': [
        'translation_request', 'language_learning', 'cultural_context',
        'grammar_check', 'pronunciation_help', 'localization'
    ],
    'general': [
        'greeting', 'goodbye', 'thank_you', 'apology', 
        'clarification_request', 'escalation_request'
    ]
}

# Création du mapping des labels
all_intents = []
for domain, intents in INTENT_CATEGORIES.items():
    all_intents.extend(intents)

intent_to_id = {intent: i for i, intent in enumerate(all_intents)}
id_to_intent = {i: intent for intent, i in intent_to_id.items()}

print(f"📊 Nombre total d'intentions: {len(all_intents)}")
print(f"🏷️ Intentions par domaine:")
for domain, intents in INTENT_CATEGORIES.items():
    print(f"  - {domain}: {len(intents)} intentions")

In [ ]:
# Génération de données d'exemple (en attendant les vraies données)
def generate_sample_data(num_samples_per_intent: int = 50) -> List[Dict]:
    """Génère des données d'exemple pour chaque intention."""
    
    sample_texts = {
        # Technical
        'code_debug': [
            "Mon code Python ne fonctionne pas, pouvez-vous m'aider ?",
            "J'ai une erreur dans ma fonction, comment la corriger ?",
            "Help me debug this JavaScript issue",
            "There's a bug in my React component"
        ],
        'documentation_request': [
            "Où puis-je trouver la documentation de cette API ?",
            "Can you show me the docs for this library?",
            "I need help understanding this framework",
            "Comment utiliser cette bibliothèque ?"
        ],
        # E-commerce
        'product_search': [
            "Je cherche un ordinateur portable pour le gaming",
            "Show me smartphones under $500",
            "Quels sont vos meilleurs téléphones ?",
            "I'm looking for wireless headphones"
        ],
        'price_inquiry': [
            "Quel est le prix de ce produit ?",
            "How much does this cost?",
            "Is there a discount available?",
            "Y a-t-il des promotions en cours ?"
        ],
        # HR
        'job_inquiry': [
            "Quels postes sont disponibles ?",
            "Are there any developer positions open?",
            "I'm interested in working at your company",
            "Comment postuler pour un emploi ?"
        ],
        'interview_preparation': [
            "Comment me préparer pour l'entretien ?",
            "What questions should I expect?",
            "Tips for a technical interview?",
            "Conseils pour réussir l'entretien"
        ],
        # Multilingual
        'translation_request': [
            "Pouvez-vous traduire ce texte en anglais ?",
            "Translate this to French please",
            "¿Puedes traducir esto al español?",
            "この文を英語に翻訳してください"
        ],
        # General
        'greeting': [
            "Bonjour !", "Hello!", "Hi there", "Salut",
            "Good morning", "Bonsoir", "Hey", "Hola"
        ],
        'thank_you': [
            "Merci beaucoup", "Thank you", "Thanks a lot",
            "Je vous remercie", "Much appreciated", "Gracias"
        ]
    }
    
    data = []
    
    for intent in all_intents:
        if intent in sample_texts:
            base_samples = sample_texts[intent]
        else:
            # Génération de textes génériques pour les intentions non définies
            base_samples = [
                f"This is about {intent.replace('_', ' ')}",
                f"I need help with {intent.replace('_', ' ')}",
                f"Question about {intent.replace('_', ' ')}",
                f"How to handle {intent.replace('_', ' ')}?"
            ]
        
        # Augmentation des données
        for i in range(num_samples_per_intent):
            text = np.random.choice(base_samples)
            # Ajout de variations
            if np.random.random() > 0.7:
                prefixes = ["Could you help me with ", "I have a question about ", ""]
                text = np.random.choice(prefixes) + text.lower()
            
            data.append({
                'text': text,
                'intent': intent,
                'label': intent_to_id[intent],
                'domain': next(domain for domain, intents in INTENT_CATEGORIES.items() if intent in intents)
            })
    
    return data

# Génération des données
print("🔄 Génération des données d'exemple...")
raw_data = generate_sample_data(num_samples_per_intent=75)
df = pd.DataFrame(raw_data)

print(f"📈 Dataset créé: {len(df)} échantillons")
print(f"📊 Répartition par domaine:")
print(df['domain'].value_counts())

# Affichage d'exemples
print("\n📝 Exemples de données:")
sample_data = df.groupby('domain').head(2)
for _, row in sample_data.iterrows():
    print(f"  {row['domain']} | {row['intent']} | '{row['text']}'")

## 📊 Analyse Exploratoire des Données

In [ ]:
# Statistiques générales
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribution par domaine
df['domain'].value_counts().plot(kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('Distribution par Domaine')
axes[0,0].set_xlabel('Domaine')
axes[0,0].set_ylabel('Nombre d\'échantillons')
axes[0,0].tick_params(axis='x', rotation=45)

# Longueur des textes
df['text_length'] = df['text'].str.len()
df['text_length'].hist(bins=30, ax=axes[0,1], color='lightcoral')
axes[0,1].set_title('Distribution de la Longueur des Textes')
axes[0,1].set_xlabel('Longueur (caractères)')
axes[0,1].set_ylabel('Fréquence')

# Top 10 des intentions
top_intents = df['intent'].value_counts().head(10)
top_intents.plot(kind='barh', ax=axes[1,0], color='lightgreen')
axes[1,0].set_title('Top 10 des Intentions')
axes[1,0].set_xlabel('Nombre d\'échantillons')

# Nombre de mots par domaine
df['word_count'] = df['text'].str.split().str.len()
df.boxplot(column='word_count', by='domain', ax=axes[1,1])
axes[1,1].set_title('Distribution du Nombre de Mots par Domaine')
axes[1,1].set_xlabel('Domaine')
axes[1,1].set_ylabel('Nombre de mots')

plt.tight_layout()
plt.show()

# Statistiques descriptives
print("📈 Statistiques descriptives:")
print(f"Longueur moyenne des textes: {df['text_length'].mean():.1f} caractères")
print(f"Nombre moyen de mots: {df['word_count'].mean():.1f}")
print(f"Échantillons les plus courts: {df['text_length'].min()} caractères")
print(f"Échantillons les plus longs: {df['text_length'].max()} caractères")

## 🔧 Préparation des Données et Tokenisation

In [ ]:
# Configuration du modèle
MODEL_NAME = "distilbert-base-multilingual-cased"
MAX_LENGTH = 128
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_EPOCHS = 5

print(f"🤖 Modèle: {MODEL_NAME}")
print(f"📏 Longueur maximale: {MAX_LENGTH}")
print(f"🎯 Nombre de classes: {len(all_intents)}")

# Chargement du tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
print(f"✅ Tokenizer chargé: {len(tokenizer)} tokens dans le vocabulaire")

# Division train/validation/test
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['intent'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['intent'])

print(f"📊 Répartition des données:")
print(f"  - Train: {len(train_df)} échantillons ({len(train_df)/len(df)*100:.1f}%)")
print(f"  - Validation: {len(val_df)} échantillons ({len(val_df)/len(df)*100:.1f}%)")
print(f"  - Test: {len(test_df)} échantillons ({len(test_df)/len(df)*100:.1f}%)")

In [ ]:
def preprocess_function(examples):
    """Fonction de préprocessing pour la tokenisation."""
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )

# Conversion en datasets Hugging Face
train_dataset = Dataset.from_pandas(train_df[['text', 'label']].reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df[['text', 'label']].reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df[['text', 'label']].reset_index(drop=True))

# Tokenisation
print("🔄 Tokenisation en cours...")
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Configuration des colonnes
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print("✅ Tokenisation terminée")
print(f"📝 Exemple tokenisé:")
sample = train_dataset[0]
print(f"  Input IDs shape: {sample['input_ids'].shape}")
print(f"  Attention mask shape: {sample['attention_mask'].shape}")
print(f"  Label: {sample['label']} ({id_to_intent[sample['label'].item()]})")

## 🧠 Configuration et Entraînement du Modèle

In [ ]:
# Chargement du modèle
model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(all_intents),
    problem_type="single_label_classification"
)

print(f"🤖 Modèle chargé: {sum(p.numel() for p in model.parameters()):,} paramètres")
print(f"🎯 Nombre de classes de sortie: {model.num_labels}")

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir='../models/intent_classifier',
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=LEARNING_RATE,
    logging_dir='../logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
    report_to=None,  # Désactiver wandb pour ce notebook
    seed=42
)

print("⚙️ Configuration d'entraînement:")
print(f"  - Épochs: {NUM_EPOCHS}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Warmup steps: {training_args.warmup_steps}")
print(f"  - Weight decay: {training_args.weight_decay}")

In [ ]:
# Métriques d'évaluation
def compute_metrics(eval_pred):
    """Calcule les métriques d'évaluation."""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    # Calcul des métriques
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Configuration du trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("✅ Trainer configuré avec early stopping")

In [ ]:
# Entraînement du modèle
print("🚀 Début de l'entraînement...")
print(f"⏱️ Temps estimé: ~{NUM_EPOCHS * len(train_dataset) // BATCH_SIZE // 60} minutes")

# Démarrage de l'entraînement
trainer.train()

print("🎉 Entraînement terminé !")

# Sauvegarde du meilleur modèle
trainer.save_model('../models/intent_classifier_best')
tokenizer.save_pretrained('../models/intent_classifier_best')

print("💾 Modèle sauvegardé dans ../models/intent_classifier_best")

## 📈 Évaluation et Métriques

In [ ]:
# Évaluation sur l'ensemble de test
print("🧪 Évaluation sur l'ensemble de test...")
test_results = trainer.evaluate(test_dataset)

print("📊 Résultats finaux:")
for metric, value in test_results.items():
    if 'eval_' in metric:
        clean_metric = metric.replace('eval_', '')
        if isinstance(value, float):
            print(f"  {clean_metric.capitalize()}: {value:.4f}")

# Prédictions détaillées
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

# Confidence scores
confidence_scores = torch.softmax(torch.tensor(predictions.predictions), dim=1)
max_confidence = torch.max(confidence_scores, dim=1)[0].numpy()

print(f"\n🎯 Confidence moyenne: {np.mean(max_confidence):.4f}")
print(f"📊 Distribution des confidence scores:")
print(f"  - > 0.9: {np.sum(max_confidence > 0.9) / len(max_confidence) * 100:.1f}%")
print(f"  - 0.7-0.9: {np.sum((max_confidence > 0.7) & (max_confidence <= 0.9)) / len(max_confidence) * 100:.1f}%")
print(f"  - < 0.7: {np.sum(max_confidence <= 0.7) / len(max_confidence) * 100:.1f}%")

In [ ]:
# Rapport de classification détaillé
intent_names = [id_to_intent[i] for i in range(len(all_intents))]
classification_rep = classification_report(
    y_true, y_pred, 
    target_names=intent_names,
    output_dict=True
)

# Conversion en DataFrame pour une meilleure visualisation
class_df = pd.DataFrame(classification_rep).transpose()
class_df = class_df.sort_values('f1-score', ascending=False)

print("🏆 Top 10 des meilleures intentions (F1-score):")
top_10 = class_df.head(10)
for intent, row in top_10.iterrows():
    if intent not in ['accuracy', 'macro avg', 'weighted avg']:
        print(f"  {intent}: F1={row['f1-score']:.3f}, Précision={row['precision']:.3f}, Rappel={row['recall']:.3f}")

print("\n⚠️ Intentions les plus difficiles (F1-score):")
bottom_5 = class_df.tail(8).head(5)  # Éviter les moyennes
for intent, row in bottom_5.iterrows():
    if intent not in ['accuracy', 'macro avg', 'weighted avg']:
        print(f"  {intent}: F1={row['f1-score']:.3f}, Précision={row['precision']:.3f}, Rappel={row['recall']:.3f}")

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)

# Visualisation de la matrice de confusion (version simplifiée par domaine)
domain_mapping = {}
for intent in all_intents:
    for domain, intents in INTENT_CATEGORIES.items():
        if intent in intents:
            domain_mapping[intent] = domain
            break

# Agrégation par domaine
y_true_domain = [domain_mapping[id_to_intent[label]] for label in y_true]
y_pred_domain = [domain_mapping[id_to_intent[pred]] for pred in y_pred]

domain_cm = confusion_matrix(y_true_domain, y_pred_domain, labels=list(INTENT_CATEGORIES.keys()))

# Visualisation
plt.figure(figsize=(10, 8))
sns.heatmap(domain_cm, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=list(INTENT_CATEGORIES.keys()),
            yticklabels=list(INTENT_CATEGORIES.keys()))
plt.title('Matrice de Confusion par Domaine')
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.tight_layout()
plt.show()

# Accuracy par domaine
domain_accuracy = {}
for domain in INTENT_CATEGORIES.keys():
    domain_mask = [d == domain for d in y_true_domain]
    if sum(domain_mask) > 0:
        domain_true = [y_true_domain[i] for i, mask in enumerate(domain_mask) if mask]
        domain_pred = [y_pred_domain[i] for i, mask in enumerate(domain_mask) if mask]
        domain_accuracy[domain] = accuracy_score(domain_true, domain_pred)

print("🎯 Accuracy par domaine:")
for domain, acc in sorted(domain_accuracy.items(), key=lambda x: x[1], reverse=True):
    print(f"  {domain}: {acc:.3f}")

## 🧪 Tests et Démonstration

In [ ]:
def predict_intent(text: str, model, tokenizer, top_k: int = 3):
    """Prédit l'intention d'un texte avec les top-k prédictions."""
    
    # Tokenisation
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=MAX_LENGTH
    )
    
    # Prédiction
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.softmax(outputs.logits, dim=1)
    
    # Top-k prédictions
    top_k_probs, top_k_indices = torch.topk(probabilities, top_k)
    
    results = []
    for i in range(top_k):
        intent_id = top_k_indices[0][i].item()
        intent_name = id_to_intent[intent_id]
        confidence = top_k_probs[0][i].item()
        domain = domain_mapping[intent_name]
        
        results.append({
            'intent': intent_name,
            'domain': domain,
            'confidence': confidence
        })
    
    return results

# Tests avec des exemples
test_examples = [
    "Mon code Python ne marche pas, peux-tu m'aider ?",
    "Je cherche un téléphone pas cher",
    "Bonjour, comment ça va ?",
    "Quels sont les postes disponibles dans votre entreprise ?",
    "Can you translate this text to French?",
    "I need help with debugging my JavaScript",
    "What's the price of this laptop?",
    "Merci beaucoup pour votre aide",
    "How do I prepare for a technical interview?",
    "¿Puedes ayudarme con la gramática española?"
]

print("🧪 Tests de prédiction d'intentions:\n")
for i, text in enumerate(test_examples, 1):
    print(f"📝 Test {i}: '{text}'")
    predictions = predict_intent(text, model, tokenizer, top_k=3)
    
    print("🎯 Prédictions:")
    for j, pred in enumerate(predictions, 1):
        print(f"  {j}. {pred['intent']} ({pred['domain']}) - {pred['confidence']:.3f}")
    print()

## 📈 Analyse des Erreurs et Améliorations

In [ ]:
# Analyse des erreurs les plus fréquentes
error_analysis = []

for i, (true_label, pred_label, confidence) in enumerate(zip(y_true, y_pred, max_confidence)):
    if true_label != pred_label:
        error_analysis.append({
            'text': test_df.iloc[i]['text'],
            'true_intent': id_to_intent[true_label],
            'pred_intent': id_to_intent[pred_label],
            'true_domain': domain_mapping[id_to_intent[true_label]],
            'pred_domain': domain_mapping[id_to_intent[pred_label]],
            'confidence': confidence,
            'cross_domain': domain_mapping[id_to_intent[true_label]] != domain_mapping[id_to_intent[pred_label]]
        })

error_df = pd.DataFrame(error_analysis)

print(f"❌ Nombre total d'erreurs: {len(error_df)} sur {len(y_true)} ({len(error_df)/len(y_true)*100:.1f}%)")

if len(error_df) > 0:
    print(f"🔄 Erreurs inter-domaines: {error_df['cross_domain'].sum()} ({error_df['cross_domain'].sum()/len(error_df)*100:.1f}%)")
    print(f"📊 Confidence moyenne des erreurs: {error_df['confidence'].mean():.3f}")
    
    print("\n🔍 Erreurs les plus fréquentes:")
    error_patterns = error_df.groupby(['true_intent', 'pred_intent']).size().sort_values(ascending=False).head(10)
    for (true_intent, pred_intent), count in error_patterns.items():
        print(f"  {true_intent} → {pred_intent}: {count} fois")
    
    print("\n⚠️ Exemples d'erreurs avec faible confidence:")
    low_confidence_errors = error_df[error_df['confidence'] < 0.7].head(5)
    for _, row in low_confidence_errors.iterrows():
        print(f"  '{row['text']}'")
        print(f"    Vraie: {row['true_intent']} | Prédite: {row['pred_intent']} | Conf: {row['confidence']:.3f}")
        print()

## 💾 Sauvegarde et Export

In [ ]:
# Sauvegarde des métadonnées du modèle
model_metadata = {
    'model_name': MODEL_NAME,
    'num_labels': len(all_intents),
    'intent_to_id': intent_to_id,
    'id_to_intent': id_to_intent,
    'intent_categories': INTENT_CATEGORIES,
    'domain_mapping': domain_mapping,
    'max_length': MAX_LENGTH,
    'training_params': {
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'num_epochs': NUM_EPOCHS,
        'max_length': MAX_LENGTH
    },
    'performance': {
        'test_accuracy': test_results['eval_accuracy'],
        'test_f1': test_results['eval_f1'],
        'test_precision': test_results['eval_precision'],
        'test_recall': test_results['eval_recall']
    },
    'dataset_info': {
        'total_samples': len(df),
        'train_samples': len(train_df),
        'val_samples': len(val_df),
        'test_samples': len(test_df)
    }
}

# Sauvegarde des métadonnées
os.makedirs('../models/intent_classifier_best', exist_ok=True)
with open('../models/intent_classifier_best/model_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(model_metadata, f, indent=2, ensure_ascii=False)

# Sauvegarde des résultats détaillés
results_summary = {
    'classification_report': classification_rep,
    'domain_accuracy': domain_accuracy,
    'confusion_matrix': cm.tolist(),
    'error_analysis': error_df.to_dict('records') if len(error_df) > 0 else []
}

with open('../models/intent_classifier_best/evaluation_results.json', 'w', encoding='utf-8') as f:
    json.dump(results_summary, f, indent=2, ensure_ascii=False)

print("💾 Métadonnées et résultats sauvegardés dans ../models/intent_classifier_best/")
print("✅ Modèle prêt pour la production !")

# Résumé final
print("\n🎉 RÉSUMÉ FINAL:")
print(f"📊 Accuracy finale: {test_results['eval_accuracy']:.4f}")
print(f"🎯 F1-score: {test_results['eval_f1']:.4f}")
print(f"🏷️ Intentions supportées: {len(all_intents)}")
print(f"🌍 Domaines: {len(INTENT_CATEGORIES)}")
print(f"💻 Modèle: {MODEL_NAME}")
print(f"📁 Emplacement: ../models/intent_classifier_best/")